# Example of Weak Label Learning with warm up

In [1]:
import torch
import math
from datasets import openml_datasets,torch_datasets
from utils import weakener, losses, trainig_testing
from models import general_model

In [19]:
#DS = torch_datasets.Torch_Dataset('mnist')
DS = openml_datasets.OpenML_Dataset('iris',batch_size=8)
train_x, train_y, test_x, test_y = DS.get_data()

In [3]:
print(DS.num_features)
print(DS.num_classes)

4
tensor(3, dtype=torch.int32)


In [4]:
Weak = weakener.Weakener(DS.num_classes)

In [5]:
print(dir(Weak))

['M', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'c', 'generate_M', 'generate_weak', 'generate_wl_priors', 'label_matrix', 'pll_weights', 'virtual_labels', 'virtual_matrix', 'w', 'z']


In [6]:
print(DS.__dict__)

{'dataset': 'iris', 'tr_size': 0.7, 'weak_labels': None, 'batch_size': 64, 'shuffle': False, 'splitting_seed': 47, 'num_classes': tensor(3, dtype=torch.int32), 'train_dataset': <torch.utils.data.dataset.TensorDataset object at 0x0000016CFF98D550>, 'test_dataset': <torch.utils.data.dataset.TensorDataset object at 0x0000016CFF98DFA0>, 'train_num_samples': 105, 'test_num_samples': 45, 'num_features': 4}


In [7]:
Weak.generate_M(model_class='pll')
z, w  = Weak.generate_weak(train_y)

In [8]:
print(Weak.__dict__)

{'c': tensor(3, dtype=torch.int32), 'M': array([[0.   , 0.375, 0.375],
       [0.375, 0.   , 0.375],
       [0.375, 0.375, 0.   ],
       [0.25 , 0.25 , 0.25 ]]), 'z': tensor([3, 0, 3, 2, 2, 1, 2, 0, 0, 3, 2, 2, 0, 2, 1, 3, 1, 3, 2, 3, 3, 2, 0, 3,
        3, 2, 0, 0, 1, 1, 3, 2, 2, 3, 0, 0, 1, 2, 2, 1, 2, 3, 1, 0, 0, 1, 2, 2,
        3, 0, 3, 0, 3, 0, 0, 2, 1, 3, 2, 3, 1, 1, 1, 2, 0, 3, 1, 0, 1, 1, 0, 2,
        2, 1, 1, 2, 1, 0, 1, 3, 2, 0, 2, 3, 1, 0, 1, 3, 1, 0, 1, 0, 3, 1, 0, 3,
        0, 3, 0, 0, 0, 0, 3, 3, 0], dtype=torch.int32), 'w': tensor([[1., 1., 1.],
        [0., 1., 1.],
        [1., 1., 1.],
        [1., 1., 0.],
        [1., 1., 0.],
        [1., 0., 1.],
        [1., 1., 0.],
        [0., 1., 1.],
        [0., 1., 1.],
        [1., 1., 1.],
        [1., 1., 0.],
        [1., 1., 0.],
        [0., 1., 1.],
        [1., 1., 0.],
        [1., 0., 1.],
        [1., 1., 1.],
        [1., 0., 1.],
        [1., 1., 1.],
        [1., 1., 0.],
        [1., 1., 1.],
        [1.

In [13]:
#As we are first making a warm up, we will include in the dataset w
#DS.include_weak(Weak.w)

Weak.virtual_matrix()
Weak.virtual_labels(train_y)
DS.include_weak(Weak.w)

In [14]:
trainloader_wu, testloader_wu = DS.get_dataloader()

## Begin the training process

In [15]:
#This is the mlp used by Valen for [mnist,kmnist,fmnist] with random partialization (aka instance independet corruption)
mlp = general_model.MLP(DS.num_features, [DS.num_features, DS.num_features, DS.num_features], DS.num_classes, do = False, bn = False, seed = 10,
                        layer_init = lambda x: torch.nn.init.kaiming_uniform_(x, a=math.sqrt(5)))
mlp

MLP(
  (layers): ModuleList(
    (0): Linear(in_features=4, out_features=4, bias=True)
    (1): Linear(in_features=4, out_features=4, bias=True)
    (2): Linear(in_features=4, out_features=4, bias=True)
    (3): Linear(in_features=4, out_features=3, bias=True)
  )
)

In [18]:
#optim_wu  = torch.optim.SGD(mlp.parameters(), lr = 1e-2, weight_decay = 1e-4, momentum = 0.9)
optim_wu  = torch.optim.Adam(mlp.parameters(), lr = 1e-2, weight_decay = 1e-4)
loss_fn_wu = losses.OSLCELoss()
#loss_fn_wu =
mlp, results_wu = trainig_testing.train_and_evaluate(mlp, trainloader_wu, testloader_wu, optim_wu, loss_fn_wu, num_epochs = 10)




tensor([[5.6000, 2.8000, 4.9000, 2.0000],
        [5.6000, 2.9000, 3.6000, 1.3000],
        [6.7000, 2.5000, 5.8000, 1.8000],
        [6.7000, 3.0000, 5.0000, 1.7000],
        [4.4000, 2.9000, 1.4000, 0.2000],
        [5.8000, 4.0000, 1.2000, 0.2000],
        [4.3000, 3.0000, 1.1000, 0.1000],
        [4.9000, 2.4000, 3.3000, 1.0000],
        [6.8000, 3.2000, 5.9000, 2.3000],
        [4.9000, 3.0000, 1.4000, 0.2000],
        [4.6000, 3.6000, 1.0000, 0.2000],
        [5.4000, 3.4000, 1.7000, 0.2000],
        [6.1000, 2.6000, 5.6000, 1.4000],
        [5.1000, 3.3000, 1.7000, 0.5000],
        [6.9000, 3.1000, 5.1000, 2.3000],
        [7.9000, 3.8000, 6.4000, 2.0000],
        [6.7000, 3.1000, 5.6000, 2.4000],
        [5.0000, 2.3000, 3.3000, 1.0000],
        [5.2000, 3.4000, 1.4000, 0.2000],
        [5.1000, 3.8000, 1.9000, 0.4000],
        [7.3000, 2.9000, 6.3000, 1.8000],
        [5.5000, 2.4000, 3.8000, 1.1000],
        [5.5000, 2.3000, 4.0000, 1.3000],
        [4.8000, 3.0000, 1.4000, 

In [ ]:

Weak.virtual_labels(train_y)

In [ ]:
DS.include_weak(Weak.v)
trainloader, testloader = DS.get_dataloader()

In [ ]:
DS.__dict__

In [ ]:
#optim  = torch.optim.SGD(mlp.parameters(), lr = 1e-2, weight_decay = 1e-4, momentum = 0.9)
optim  = torch.optim.Adam(mlp.parameters(), lr = 1e-2, weight_decay = 1e-4)
loss_fn = losses.CELoss()

#mlp, results = trainig_testing.train_and_evaluate(mlp, trainloader, testloader, optim, loss_fn, num_epochs = 490)
mlp, results = trainig_testing.train_model(mlp, trainloader, optim, loss_fn, num_epochs = 490)